## Magentic One

![magentic](https://www.microsoft.com/en-us/research/uploads/prod/2024/11/magnetic_example-2048x1155.png)

### `magentic_one.py` core code

```python
def __init__(self, client: ChatCompletionClient, hil_mode: bool = False):
    self.client = client
    self._validate_client_capabilities(client)

    fs = FileSurfer("FileSurfer", model_client=client)
    ws = MultimodalWebSurfer("WebSurfer", model_client=client)
    coder = MagenticOneCoderAgent("Coder", model_client=client)
    executor = CodeExecutorAgent("Executor", code_executor=LocalCommandLineCodeExecutor())
    agents: List[ChatAgent] = [fs, ws, coder, executor]
    if hil_mode:
        user_proxy = UserProxyAgent("User")
        agents.append(user_proxy)
    super().__init__(agents, model_client=client)

```

In [5]:
# Autonomously complete a coding task:
import asyncio
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_ext.teams.magentic_one import MagenticOne
from autogen_agentchat.ui import Console

from dotenv import load_dotenv
import os

load_dotenv()

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model=deployment_name,
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key,
        )


client = azure_openai_chat_completion_client
m1 = MagenticOne(client=client)
task = "마이크로소프트 현재 주가와 전망에 대해 알려줘"
result = await Console(m1.run_stream(task=task))
print(result)

/var/folders/pr/hngks2mn74d25jycvx6rxl400000gn/T/ipykernel_10055/3377041901.py:25: DeprecationWarning: Instantiating MagenticOne without a code_executor is deprecated. Provide a code_executor to clear this warning (e.g., code_executor=LocalCommandLineCodeExecutor() ).
  m1 = MagenticOne(client=client)


---------- TextMessage (user) ----------
마이크로소프트 현재 주가와 전망에 대해 알려줘
---------- TextMessage (MagenticOneOrchestrator) ----------

We are working to address the following user request:

마이크로소프트 현재 주가와 전망에 대해 알려줘


To answer this request we have assembled the following team:

FileSurfer: An agent that can handle local files.
WebSurfer: A helpful assistant with access to a web browser. Ask them to perform web searches, open pages, and interact with content (e.g., clicking links, scrolling the viewport, filling in form fields, etc.). It can also summarize the entire page, or answer questions based on the content of the page. It can also be asked to sleep and wait for pages to load, in cases where the page seems not yet fully loaded.
Coder: A helpful and general-purpose AI assistant that has strong language skills, Python skills, and Linux command line skills.
ComputerTerminal: A computer terminal that performs no other action than running Python scripts (provided to it quoted in ```python co